In [1]:
import ephyviewer

import matplotlib.pyplot as plt
from pathlib import Path
import spikeinterface.full as si
from probeinterface import read_spikeglx
from probeinterface.plotting import plot_probe
import numpy as np
%matplotlib widget

In [10]:
# Paths
base_folder = Path('/home/arthur/Documents/SpikeSorting/Test_20210518/') 
data_folder = base_folder / 'raw_awake'
recording_nidq = si.SpikeGLXRecordingExtractor(data_folder, stream_id='nidq') # microphone
print(recording_nidq)

SpikeGLXRecordingExtractor: 9 channels - 1 segments - 20.0kHz - 3881.356s


In [3]:
fs = recording_nidq.get_sampling_frequency()
max_time = recording_nidq.get_total_duration()

In [4]:
trace = recording_nidq.get_traces(segment_index=0, channel_ids=['nidq#XA0'])

In [20]:
plt.plot(np.linspace(0, max_time, len(trace)), trace)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (uV)')

Text(28.222222222222214, 0.5, 'Amplitude (uV)')

In [8]:
t_start = int(1000*fs)
t_end = int(1030*fs)

roi = trace[t_start:t_end]
duration_roi = np.linspace(t_start, t_end, len(roi))

In [ ]:
plt.plot(duration_roi/fs, roi)

The benefit of a longer FFT is that you will have more frequency resolution. The number of FFT bins, the discrete fequency interval of the transform will be N/2. So the frequency resolution of each bin will be the sample frequency Fs x 2/N.
Large FFT sizes produce high frequency resolution but poor time resolution while low FFT sizes have the opposite effect.  
Overlap processing can be used to provide high resolution in both the frequency and time axis.  For example, if a signal contains two discrete tones separated in frequency, only one of which is on at a time, it is difficult to discern the relative timing of each tone because both frequencies are often contained within a single FFT size time series block.  If we perform an FFT and, instead of moving forward a whole FFT size block in the series, move forward only a small amount then we can isolate the pulse timing for each tone. 

In [9]:
from matplotlib.ticker import FormatStrFormatter
fig, ax = plt.subplots(figsize=(15,5))
NFFT = 512
spec, freq, t, im = plt.specgram(roi.ravel(),
            Fs=fs, 
            NFFT=NFFT, 
            noverlap=NFFT*0.90, 
            scale_by_freq=True, 
            mode="psd", 
            pad_to=915, 
            cmap="inferno", 
            vmin=-30 , 
            vmax=40,
            scale='dB',
            detrend='mean')
# plt.xticks(duration_roi, duration_roi, rotation='vertical')
ax.set_ylabel("Frequency (Hz)")
ax.locator_params(axis="x", nbins=8)
ticks = ax.get_xticks().tolist()
new_ticks = ["%.2f"%item for item in np.linspace(duration_roi[0], duration_roi[-1], len(ticks))/fs]
ax.set_xticklabels(new_ticks)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-9-7b381c69d823>:21: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(new_ticks)


[Text(0.0, 0, '1000.00'),
 Text(5.0, 0, '1005.00'),
 Text(10.0, 0, '1010.00'),
 Text(15.0, 0, '1015.00'),
 Text(20.0, 0, '1020.00'),
 Text(25.0, 0, '1025.00'),
 Text(30.0, 0, '1030.00')]

In [ ]:
sig_source2 = ephyviewer.FromSpikeinterfaceRecordingSource(recording=recording_nidq) # microphone

In [ ]:
# App and viewer objects
app = ephyviewer.mkQApp()
win = ephyviewer.MainViewer(debug=True, show_auto_scale=True)
view2 = ephyviewer.SpecgramViewer(source=sig_source2, name='signals nidq') # Trace of Microphone
win.add_view(view2)

In [ ]:
# Display
win.show()
app.exec_()

In [ ]:
ephyviewer.SpecgramViewer